In [ ]:
Where are we currently in the repository?

In [1]:
!pwd

/home/jgutman/mvesc/Descriptives/cohort_analysis


In [2]:
!git branch

  clean_grades_and_attendance_tables
* cohort_analysis
  consolidating_tables
  import_text_files_issue1
  master
  track_students_descriptives


In [3]:
!ls /home/jgutman/mvesc/ETL

all_absences_generate_mm_day_wkd.sql  etl_test_write_ZZ.ipynb
building_cohort_analysis.sql	      file_to_table_name.json
build_student_tracking.py	      fill_missing_years.py
clean_absences.sql		      module_dev_utility_functions.ipynb
clean_oaaogt_0616.sql		      mvesc_utility_functions.py
csv2postgres_mvesc.py		      __pycache__
ETL_test_and_dev_XC.ipynb	      snapshot_column_names.json


In [4]:
import os, sys, imp
parentdir = os.path.abspath('/home/jgutman/mvesc/ETL')
sys.path.insert(0,parentdir) 

In [5]:
import build_student_tracking as track
from mvesc_utility_functions import postgres_pgconnection_generator

In [11]:
imp.reload(track)

<module 'build_student_tracking' from '/home/jgutman/mvesc/ETL/build_student_tracking.py'>

In [12]:
with postgres_pgconnection_generator() as connection:
        with connection.cursor() as cursor:
            # print(track.sql_gen_tracking_students(2006, 2015))
            # print(track.cohort_survival_analysis(2006, 2015, '04'))
            print(track.build_wide_format(cursor, '09', 2010))
            get_graduates_query = """
        select student_lookup, graduation_date from 
            (select student_lookup, max(school_year) as school_year from clean.all_snapshots 
                where student_lookup in 
                    (select distinct student_lookup from clean.all_snapshots 
                        where graduation_date is not null) 
                and graduation_date is not null
            group by student_lookup) as latest_grade_with_graduation
        left join 
            (select student_lookup, school_year, graduation_date from clean.all_snapshots
                where graduation_date is not null) as graduation_dates_valid
        using (student_lookup, school_year)
            """
            cursor.execute(get_graduates_query)
            graduates = cursor.fetchall()
            print(len(graduates))
        connection.commit()

['grade', 'school_year', 'count']
[(9, 2010, 1397), (10, 2011, 1034), (11, 2012, 964), (12, 2013, 923)]
9222


In [18]:
import pandas as pd
pd.__version__

'0.18.1'

In [17]:
import pydot
pydot.__version__

'1.2.2'